In [1]:
from IPython.display import Image # For visualisation of image files in Jupyter notebook

from boolnetlab import BN_Realisation as BN

ModuleNotFoundError: No module named 'boolnetlab'

This is how a random Boolean network can be generated:

In [ ]:
bn_random = BN.generate_random_bn(num_nodes = 5, max_parent_nodes = 2, min_parent_nodes = 0, allow_self_loops = True, allow_input_nodes = True, mode = 'asynchronous')

and saved to a file in the ISPL format:

In [ ]:
bn_random.save_ispl('saved_models/bn_random.ispl')

Let's load a model of 15 nodes: 

In [ ]:
bn = BN.load_ispl('models/bn_7.ispl')

Plot structure (node dependency) graph of the Boolean network with the use of specified layout. The possible layouts are: 'spring' (default), 'circular', 'forceatlas', 'planar', 'random', 'shell', and 'spectral'.

In [ ]:
bn.plot_structure_graph()

In [ ]:
bn.plot_structure_graph('planar')

Plot the structure graph with PyGraphviz and save it to a file:

In [ ]:
filepath = 'aux/bn_7.png'
bn.plot_structure_graph_pgv(filepath, layout='dot')
Image(filename=filepath)

Make an interactive graph of the BN structure in HTML format. Once the generated HTML file is opened in a web browser:
- the nodes can be moved around,
- howering over edges displays the information on the Boolean function regulating the target node,
- the layout algorithm can be chosen from the 'solver' dropdown list in the 'physics' section below, which also allows to set the specific parameters of the layout algorithm,
- the image can be saved by right-clicking on the graph in the web browser and selecting "Save image as ...".

In [ ]:
filepath = 'aux/bn_7_interactive.html'
bn.plot_structure_graph_interactive(filepath)

Let us now compute the attractors:

In [ ]:
bn_attractors = bn.find_all_attractors()

The method returns a list of Binary Decision Diagrams, each encoding a set of attractor states. The BDD-encoded attractors can be explicitly enumerated with the _enumerate_attractors() method returning a dictionary where keys are strings of the form 'Ai', where i is a consecutive number identifying each attractor (the numbering starts from 0), and each value is a set of attractor states.    

In [ ]:
attractor_states = bn._enumerate_attractors(bn_attractors)
print(attractor_states)

The state transision graph is computed with a brute-force approach and the computations may take a while. The visualisation makes sense for rather small networks of size not more than 10 nodes.

In [ ]:
filepath = 'aux/bn_7_dynamics.png'
bn.draw_state_transition_graph(filepath, highlight_attractors=False)
Image(filename=filepath)

We can color the attractors in the state transition system:

In [ ]:
filepath = 'aux/bn_7_dynamics.png'
bn.draw_state_transition_graph(filepath, highlight_attractors=True, use_bdds=False, transient_state_color='chartreuse')
Image(filename=filepath)

Or we can choose our own colors:

In [ ]:
filepath = 'aux/bn_7_dynamics.png'
bn.draw_state_transition_graph(filepath, highlight_attractors=True, use_bdds=True, color_names=['red', 'green'], transient_state_color='white')
Image(filename=filepath)

Let's now compute the attractors reachable from a list of specified states:

In [ ]:
#reachable_attractors = bn.get_reachable_attractors_with_bdd([(0,0,0,1,1,0,0)])
bn = BN.load_ispl('models/bn_7.ispl')

state = (0,0,0,0,0,0,0)
reachable_attractors = bn.get_reachable_attractors_with_bdd([state])
print(reachable_attractors)
print(f"Attractor size: {len(reachable_attractors['A0'])}")

The next step is to see how removing the edge x1 -> x5 in the structure graph of the network for 2 steps will influence the reachability of attractors:

In [ ]:
bn = BN.load_ispl('models/bn_7.ispl')

nsteps = 2

reachable_attractors_er = bn.get_reachable_attractors_with_edge_removed('x1', 'x5', nsteps, state)
print(reachable_attractors_er)


In [ ]:
import utils

for i in range(2**len(bn.node_names)):
    state  = utils.int2bin(i, len(bn.node_names))
    state = tuple([0 if c == '0' else 1 for c in state])
    print(f"--- {i} ---")
    print(state)

    #bn = BN.load_ispl('models/bn_7.ispl')
    reachable_attractors_1 = bn.get_reachable_attractors_with_bdd([state])
    reachable_attractors_2 = bn.get_reachable_attractors_with_edge_removed('x1', 'x5', nsteps, state)

    a1 = reachable_attractors_1['A0']
    a2 = reachable_attractors_2['A0']

    if len(reachable_attractors_1) != len(reachable_attractors_2):
        print("Numbers of attractors differ")
        print(reachable_attractors_1)
        print(reachable_attractors_2)
    elif a1 != a2:
        print("Attractors differ")
        print(f"Size 1: {len(a1)}, Size 2: {len(a2)} ")
        print(a1.difference(a2))
        print(a2.difference(a1))

    print("---------")



We will now use BoolNetLab to verify the obtained result. For this purpose, we will draw a state transition graph of the original Boolean network with highlighted (with the yellow color) states that are reachable with the specified number of steps (nsteps) with the modified network dynamics. By visually inspecting the graph, one can see that indeed only the attractors returned by the get_reachable_attractors_with_edge_removed() method are reachable from the highlighted states: there is no path from any of the yellow states to the two-state attractor.

In [ ]:
# Let us see how the function is changed
new_fun = bn.remove_edge('x1', 'x5', modify_model=False)
print(new_fun)
bn_node_names = bn.node_names
bn_funs = bn.functions_str
bn_funs[3] = new_fun
bn_modified = BN(bn_node_names, bn_funs)
reachable_states = list(bn_modified.get_reachable_states_in_n_steps([state], num_steps=nsteps))

filepath = 'aux/bn_7_dynamics_modified.png'
bn.draw_state_transition_graph(filepath,
                                highlight_attractors=True,
                                use_bdds=True,
                                color_names=['red', 'green'],
                                transient_state_color='white',
                                selected_state_groups=[reachable_states],
                                selected_group_colors=['yellow'])
Image(filename=filepath)


Finally, let us use the Monte Carlo method to search for the attractors.

In [ ]:
MC_attractors = bn.getAttractorsMonteCarlo(burn_in_len=1000, history_len=50000)
print(MC_attractors)

Let us check the accuracy of the Monte Carlo approach:

In [ ]:
true_attractors = bn._enumerate_attractors(bn.find_all_attractors())

true_attractor_states = set()
for attractor in true_attractors.values():
    true_attractor_states = true_attractor_states.union(attractor)

MC_attractor_states = set()
for attractor in MC_attractors.values():
    MC_attractor_states = MC_attractor_states.union(attractor)

false_positives = MC_attractor_states.difference(true_attractor_states)
true_positives = MC_attractor_states.intersection(true_attractor_states)

print(f"False positive attractor states: {false_positives}")
print(f"True positive attractor states: {true_positives}")

filepath = 'aux/bn_7_dynamics_modified.png'
bn.draw_state_transition_graph(filepath,
                                highlight_attractors=True,
                                use_bdds=True,
                                color_names=['red', 'green'],
                                transient_state_color='white',
                                selected_state_groups=[true_positives, false_positives],
                                selected_group_colors=['palegreen', 'pink']
                                )
Image(filename=filepath)


